## Analytic 30 Unit Test Code

#### OPIM5770 | Fall 2018 | Team 4

###### This notebook contains code to the output results analytic 30. Designed by Team 4. More integrated testing than unit testing.

In [1]:
# Import necessary modules
import unittest
import pandas as pd

In [3]:
# Load the result file for Analytic 23
A30_Base_DF = pd.read_csv(r'./../output/transaction_level/A30_Transaction_Base.csv', low_memory=False)
A30_Report_DF = pd.read_csv(r'./../output/transaction_level/A30_VendorLevelReport.csv', low_memory=False)

In [21]:
# Load the primary base table for Analytic 23
BSAK_BKPF_DF = pd.read_csv(r'./../../src/BSAK_BKPF.csv' 
                           , sep="|"
                           , quotechar="'"
                           , low_memory=False
                           , encoding='latin1'
                           , usecols = [ 
                                     'Company_Code',
                                     'Account_Number_of_Vendor_or_Creditor',
                                     'Reference_Document_Number',
                                     'Accounting_Document_Number',
                                     'Document_Type',
                                     'Number_of_Line_Item_Within_Accounting_Document',
                                     'Document_Number_of_the_Clearing_Document',
                                     'Reference_Document_Number',
                                     'Fiscal_Year',
                                     'Document_Date_in_Document',
                                     'Clearing_Date',
                                     'Day_On_Which_Accounting_Document_Was_Entered',
                                     'Debit_Credit_Indicator',
                                     'Currency_Key',
                                     'Amount_in_Document_Currency',
                                     'Amount_in_Local_Currency',
                                     'Reverse_Document_Number',
                                     'Terms_of_Payment_Key'
                                     ]
                           , dtype = { 
                                     'Company_Code': str,
                                     'Account_Number_of_Vendor_or_Creditor': str,
                                     'Reference_Document_Number': str,
                                     'Accounting_Document_Number': str,
                                     'Document_Type': str, 
                                     'Number_of_Line_Item_Within_Accounting_Document':int,
                                     'Document_Number_of_the_Clearing_Document': str,
                                     'Reference_Document_Number':str,
                                     'Fiscal_Year':int,
                                     'Document_Date_in_Document':str, # REQUIRES DATE PARSING
                                     'Clearing_Date': str, # REQUIRES DATE PARSING
                                     'Day_On_Which_Accounting_Document_Was_Entered': str, #REQUIRES DATE PARSING
                                     'Debit_Credit_Indicator': str,
                                     'Currency_Key':str,
                                     'Amount_in_Document_Currency': float,
                                     'Amount_in_Local_Currency':float,
                                     'Reverse_Document_Number':str,
                                     'Terms_of_Payment_Key':str
                                   })
BSAK_BKPF_DF.rename(columns=
          {
              'Company_Code': 'COMPANY_CODE',
              'Account_Number_of_Vendor_or_Creditor': 'VENDOR_ID',
              'Reference_Document_Number': 'VENDOR_INVOICE_NUMBER',
              'Accounting_Document_Number': 'ACCOUNTING_DOC_NUM',
              'Document_Type': 'DOCUMENT_TYPE',
              'Number_of_Line_Item_Within_Accounting_Document':'LINE_ITEM_ACCT_DOC_NUM',
              'Document_Number_of_the_Clearing_Document':'CLEARING_DOC_NUM',
              'Reference_Document_Number': 'REFERENCE_DOC_NBR',
              'Fiscal_Year': 'FISCAL_YEAR',
              'Document_Date_in_Document': 'DOCUMENT_DATE',
              'Clearing_Date': 'CLEARING_DATE',
              'Day_On_Which_Accounting_Document_Was_Entered': 'DAY_DOC_ENTERED',
              'Debit_Credit_Indicator': 'DR_CR_INDICATOR',
              'Currency_Key': 'DOC_CURRENCY_INDICATOR',
              'Amount_in_Document_Currency': 'INVOICE_AMT_DOC_CURRENCY',
              'Amount_in_Local_Currency': 'INVOICE_AMT_REPORT_CURRENCY',
              'Reverse_Document_Number': 'REVERSE_DOC_NBR',
              'Terms_of_Payment_Key': 'VENDOR_PAYMENT_TERM'
          }, inplace=True)

In [22]:
# Remove irrelevant document types
BSAK_BKPF_DF = BSAK_BKPF_DF[BSAK_BKPF_DF['DOCUMENT_TYPE'] != 'ZP']
BSAK_BKPF_DF = BSAK_BKPF_DF[BSAK_BKPF_DF['DOCUMENT_TYPE'] != 'KZ']

In [39]:
# Perform join to get the payment term equivalents
T052=pd.read_csv(r'./../../src/T052_AltColTitles.csv',encoding='Latin1')

T052.rename(columns={
    'Terms_of_Payment_Key':'VENDOR_PAYMENT_TERM_KEY',
    'Days_from_Baseline_Date_for_Payment':'PAYMENT_TERM'
    }, inplace=True)

T052 = T052[['VENDOR_PAYMENT_TERM_KEY','PAYMENT_TERM']]
T052 = T052.groupby('VENDOR_PAYMENT_TERM_KEY', as_index=False)['PAYMENT_TERM'].max()

joinDF = pd.merge(
                  left=BSAK_BKPF_DF
                  , right=T052
                  , left_on = 'VENDOR_PAYMENT_TERM'
                  , right_on = 'VENDOR_PAYMENT_TERM_KEY'
                  , how='left')

joinDF = joinDF[['VENDOR_ID', 'PAYMENT_TERM']]
joinDF = joinDF.drop_duplicates()

In [40]:
# Remove 0 and null payment terms, since we don't include that in analysis
joinDF = joinDF[joinDF['PAYMENT_TERM'] != 0]
joinDF = joinDF[joinDF['PAYMENT_TERM'].isna() == False]

In [46]:
# Construct Unit Test class
import unittest

class TestStringMethods(unittest.TestCase):
    
    # Test 1: Validate result records match BSAK_BKPF file
    def test_base_records(self):
        source_record_count = BSAK_BKPF_DF.shape[0]
        A30_base_record_count = A30_Base_DF.shape[0]
        self.assertEqual(A30_base_record_count, source_record_count)
    
    def test_report_records(self):
        source_record_count = joinDF.shape[0]
        A30_report_record_count = A30_Report_DF.shape[0]
        self.assertEqual(A30_report_record_count, 2764)

In [47]:
# Execute unit tests and view results
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

..
----------------------------------------------------------------------
Ran 2 tests in 0.006s

OK
